#### Imports

In [1]:
import os
import math
import time
import json
import random
import numpy as np
import pandas as pd

from tqdm import tqdm

from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import commonteamroster, commonplayerinfo

#### Step 1) Fetch Latest / Last Season Info For Each Player

In [24]:
# read data in
ballhog_df = pd.read_csv('data/ball-hog-rate/ball-hog-rates_regular_season_2013_23.csv').drop(columns=['Points', 'FG2M', 'FG2A', 'FG3M', 'FG3A', 'FtPoints',
       'PtsAssisted2s', 'PtsUnassisted2s', 'PtsAssisted3s', 'PtsUnassisted3s',
       'PtsPutbacks',])

# sort dataframe by season in ascending order, group dataframe by player_id and select first row of each group
ballhog_df = ballhog_df.sort_values('SEASON', ascending=False).reset_index(drop=True)
latest_season_df = ballhog_df.groupby('PLAYER_ID').first().reset_index()

print("The Number of Players to find their Age & Experience in their latest/last season played:", len(latest_season_df))

important_cols = ['SEASON','PLAYER_ID','PLAYER_NAME','TEAM_ID']
latest_season_df = latest_season_df[important_cols].copy()

latest_season_df['AGE'] = ''
latest_season_df['EXP'] = ''

latest_season_df.head()

The Number of Players to find their Age & Experience in their latest/last season played: 1267


,SEASON,PLAYER_ID,PLAYER_NAME,TEAM_ID,AGE,EXP
0,2015-16,708.0,Kevin Garnett,1610612750,,
1,2013-14,951.0,Ray Allen,1610612748,,
2,2013-14,959.0,Steve Nash,1610612747,,
3,2013-14,965.0,Derek Fisher,1610612760,,
4,2015-16,977.0,Kobe Bryant,1610612747,,


#### Step 2) Find traded players, and get the TEAM_ID / TEAM ABBREVIATION for their latest/last team they've been on - this is because commonteamroster for any team only provides info on players who ended the season with them

In [3]:
traded_players = pd.read_csv('data/ids/traded_players_database.csv')

# get list of teams and sort them
team_lst = teams.get_teams()
sorted_team_lst = sorted(team_lst, key=lambda x: x['full_name'])

abb_to_id_map = dict()

# map values to each other
for team in sorted_team_lst:
    abb_to_id_map[team['abbreviation']] = team['id']
    
# add team IDs to traded players database
traded_players['TEAM_ID'] = traded_players['TEAM'].map(abb_to_id_map)

for index, pid, team_id, season in zip(list(latest_season_df.index.values), latest_season_df['PLAYER_ID'].to_list(), 
                                       latest_season_df['TEAM_ID'].to_list(), latest_season_df['SEASON'].to_list()):
    player_df = ballhog_df[(ballhog_df['SEASON'] == season) & (ballhog_df['PLAYER_ID'] == pid)].reset_index(drop=True)
    if len(player_df) > 1:
        df = traded_players[(traded_players['PLAYER_ID'] == pid) & 
                            (traded_players['SEASON'] == season)].reset_index(drop=True).groupby(by=['SEASON']).last()
        new_team_id = df['TEAM_ID'].to_list()[0]

        if team_id != new_team_id:
            latest_season_df.at[index, 'TEAM_ID'] = new_team_id
            print(int(pid), team_id, '->', new_team_id, '|', index, latest_season_df.loc[index,'TEAM_ID'])
            print('--------------------------')

2440 1610612758 -> 1610612744 | 48 1610612744
--------------------------
2550 1610612741 -> 1610612737 | 58 1610612737
--------------------------
2755 1610612750 -> 1610612759 | 91 1610612759
--------------------------
101115 1610612739 -> 1610612754 | 105 1610612754
--------------------------
101183 1610612756 -> 1610612738 | 125 1610612738
--------------------------
200765 1610612747 -> 1610612739 | 139 1610612739
--------------------------
201144 1610612762 -> 1610612750 | 154 1610612750
--------------------------
201148 1610612763 -> 1610612745 | 157 1610612745
--------------------------
201189 1610612761 -> 1610612758 | 174 1610612758
--------------------------
201196 1610612752 -> 1610612764 | 175 1610612764
--------------------------
201566 1610612747 -> 1610612746 | 183 1610612746
--------------------------
201567 1610612739 -> 1610612748 | 184 1610612748
--------------------------
201571 1610612745 -> 1610612747 | 187 1610612747
--------------------------
201579 1610612766 -> 

#### Step 3) Find Age and & Experience for each Player

In [4]:
def find_age_exp_pos(team_id, season, player_id):
    '''
    returns age & experience of player's latest / last season
    '''
    attempts = 0
    
    while True:
        try:
            roster_df = commonteamroster.CommonTeamRoster(
                team_id=team_id,
                season=season,
            ).get_data_frames()[0]
            player_df = roster_df[roster_df['PLAYER_ID'] == player_id].reset_index(drop=True)

            if len(player_df) == 1:
                return player_df['AGE'].to_list()[0], player_df['EXP'].to_list()[0]

            return '',''

        except:
            
            if attempts == 40:
                return '', ''
            
            attempts += 1
            time.sleep(0.7)
        
            

In [5]:
for index, team_id, season, pid in tqdm(zip(list(latest_season_df.index.values), latest_season_df['TEAM_ID'].to_list(),
                                        latest_season_df['SEASON'].to_list(), latest_season_df['PLAYER_ID'].to_list()), 
                                        total=len(latest_season_df)):
    age, experience = find_age_exp_pos(team_id, season, int(pid))
    latest_season_df.at[index,'AGE'] = age
    latest_season_df.at[index,'EXP'] = experience
    
    time.sleep(.04)

100%|██████████████████████████████████████████████████████████████████████████████| 1267/1267 [19:51<00:00,  1.06it/s]


In [6]:
unidentified_df = latest_season_df[latest_season_df['AGE'] == ''].reset_index(drop=True)
identified_df =  latest_season_df[latest_season_df['AGE'] != ''].reset_index(drop=True)

# replace rookies with 0 years of experience
identified_df['EXP'] = identified_df['EXP'].replace('R', '0')

#### Step 4) Find Age & Experience for Players via another method (commonPlayerInfo) for the ones we couldn't find earlier

In [7]:
# Define a function to get the number of years of experience for a player
def get_latest_player_experience_age(player_id):
    '''
    return age & experience for players using commonplayerinfo method
    '''
    attempts = 0
    
    while True:
        try:
            player_df = commonplayerinfo.CommonPlayerInfo(player_id=player_id).get_data_frames()[0]
            player_df['BIRTHDATE'] = pd.to_datetime(player_df['BIRTHDATE'])

            experience = player_df['SEASON_EXP'].values[0]
            player_df['BIRTHYEAR'] = player_df['BIRTHDATE'].dt.year
            birth_year = player_df['BIRTHYEAR'].values[0]
            last_season = player_df['TO_YEAR'].values[0]
            age = last_season - birth_year
    
            return age, experience
    
        except:
            
            if attempts == 40:
                return '', ''
            
            attempts += 1
            time.sleep(0.7)

for index, pid in tqdm(zip(list(unidentified_df.index.values), unidentified_df['PLAYER_ID'].to_list()),
                             total=len(unidentified_df)):
    age, experience = get_latest_player_experience_age(int(pid))
    unidentified_df.at[index, 'AGE'] = age
    unidentified_df.at[index, 'EXP'] = experience
    time.sleep(0.05)
    
# exp was incorrectly calculated as it counted rookie year as 1 year of exp
unidentified_df['EXP'] = unidentified_df['EXP'] - 1

100%|████████████████████████████████████████████████████████████████████████████████| 245/245 [03:20<00:00,  1.22it/s]


In [8]:
identified_df['AGE'] = identified_df['AGE'].astype(np.int64)
identified_df['EXP'] = identified_df['EXP'].astype(np.int64)
unidentified_df['AGE'] = unidentified_df['AGE'].astype(np.int64)
unidentified_df['EXP'] = unidentified_df['EXP'].astype(np.int64)

In [9]:
imprt_cols = ['SEASON','PLAYER_ID','AGE','EXP']
age_exp_df = pd.concat([identified_df[imprt_cols], unidentified_df[imprt_cols]])
age_exp_df.head()

,SEASON,PLAYER_ID,AGE,EXP
0,2015-16,708.0,40,20
1,2013-14,951.0,38,17
2,2013-14,959.0,40,17
3,2013-14,965.0,39,17
4,2015-16,977.0,37,19


In [25]:
template_df = pd.merge(ballhog_df, age_exp_df, on=['SEASON','PLAYER_ID'], how='left')
template_df.head()

,SEASON,MINUTES_ON,PLAYER_NAME,PLAYER_ID,TEAM_ID,SECONDS_PER_POSS_OFFENSE_PLAYER_ON,TIME_OF_POSS,TEAM,OFF_POSS,BALL_HOG%,AGE,EXP
0,2022-23,83,A.J. Lawson,1630639.0,1610612742,14.477,3.5,DAL,230.0,6.3,22.0,0.0
1,2022-23,20,Bruno Fernando,1628981.0,1610612737,12.911,1.6,ATL,88.0,8.4,24.0,3.0
2,2022-23,754,Bryce McGowens,1631121.0,1610612766,14.117,34.7,CHA,1647.0,9.0,20.0,0.0
3,2022-23,1428,Keita Bates-Diop,1628966.0,1610612759,14.281,60.3,SAS,3043.0,8.3,27.0,4.0
4,2022-23,2356,Keegan Murray,1631099.0,1610612758,13.857,77.2,SAC,4937.0,6.8,22.0,0.0


In [11]:
updated_ballhog_df = pd.DataFrame()

for pid in tqdm(list(template_df['PLAYER_ID'].unique())):
    # get player dataframe
    player_df = template_df[template_df['PLAYER_ID'] == pid].sort_values(by=['SEASON'], ascending=False).reset_index(drop=True)

    # separate the player dataframe into rows with age/experience listed, and the other rows without it
    info_df = player_df[(~player_df['AGE'].isna()) & (~player_df['EXP'].isna())].reset_index(drop=True)
    missinginfo_df = player_df[(player_df['AGE'].isna()) & (player_df['EXP'].isna())].reset_index(drop=True)
    
  
    # get latest season, age, and experience info
    latest_season = int(info_df['SEASON'].values[0][:4])
    latest_age, latest_exp = info_df['AGE'].values[0], info_df['EXP'].values[0]
    
    if len(missinginfo_df) != 0:
        # loop through the remaining seasons in the missing info dataframe to figure out the age and experience that belongs in each
        for season in missinginfo_df['SEASON'].unique():
            
            # get dataframe for that player's season
            season_df = missinginfo_df[missinginfo_df['SEASON'] == season].reset_index(drop=True)
            test_season = int(season[:4])
            
            # every new season is an experience level below (seasons sorted in non-ascending order)
            latest_exp -= 1

            # if the season value is one less than the previous season, update player age to be one less than the previous
            if latest_season == test_season + 1:
                latest_age -= 1
                season_df['AGE'] = latest_age
                
            # else update the age such that it's year difference between the last season and this current season we're testing
            elif latest_season != test_season:
                latest_age -= (latest_season-test_season)
                season_df['AGE'] = latest_age

            # set the experience
            season_df['EXP'] = latest_exp

            # change the previous season check for next season check
            latest_season = test_season

            # add updated info to dataframe
            updated_ballhog_df = pd.concat([updated_ballhog_df, season_df]).reset_index(drop=True)
            
        # add info dataframe to dataframe
        updated_ballhog_df = pd.concat([updated_ballhog_df, info_df]).reset_index(drop=True)
        
    else:
        # add info dataframe to dataframe
        updated_ballhog_df = pd.concat([updated_ballhog_df, info_df]).reset_index(drop=True)
        
        
        # if int(season[:4])
    # print(len(info_df), latest_season, latest_age, latest_exp)

100%|█████████████████████████████████████████████████████████████████████████████| 1267/1267 [00:08<00:00, 151.04it/s]


In [12]:
wrong_exp_pid_lst = updated_ballhog_df[updated_ballhog_df['EXP'] == -1]['PLAYER_ID'].unique()

print('Row Count Check:', updated_ballhog_df.shape)

# loop through the list of pids that have wrong experience listed by (y-1)
for pid in tqdm(wrong_exp_pid_lst):
    player_df = updated_ballhog_df[updated_ballhog_df['PLAYER_ID'] == pid].reset_index(drop=True)
    updated_ballhog_df = updated_ballhog_df[~(updated_ballhog_df['PLAYER_ID'] == pid)].reset_index(drop=True)
    player_df['EXP'] = player_df['EXP'] + 1
    updated_ballhog_df = pd.concat([updated_ballhog_df, player_df]).reset_index(drop=True)
    
print('Checking to make sure we have the same number of rows as before:', updated_ballhog_df.shape)

Row Count Check: (5801, 12)


100%|█████████████████████████████████████████████████████████████████████████████████| 34/34 [00:00<00:00, 377.24it/s]

Checking to make sure we have the same number of rows as before: (5801, 12)


In [14]:
updated_ballhog_df.to_csv('data/ball-hog-rate/ball-hog-rates_regular_season_2013_23_w_age_exp.csv', index=False)

In [16]:
updated_ballhog_df[['SEASON','PLAYER_ID','AGE','EXP']].to_csv('data/ids/players_modern_database_w_age_exp.csv', index=False)